In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb
from numpy import array
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from urllib.request import urlretrieve


pd.set_option('display.max_columns', None)

In [19]:
ffile = 'C:/Users/anahe/Documents/prueba_dvc/data/full_dataset.csv' 
df = pd.read_csv(ffile, sep=",", index_col=0)
df

,CODLINEA,NOMBRELINEA,CODPEDIDO,FECHAFACTURA,IDBRICK,CODBRICK,CODARTICULO,NOMBREARTICULO,CODARTICULOSAP,CODPRODUCTO,NOMBREPRODUCTO,CODFAMILIA,NOMBREFAMILIA,VALORBASE,VALORNETO,UNIDADES,UNIDADESBONIFICADAS,UNIDADESMUESTRAS,IDTIPOPEDIDO,TIPOPEDIDO,ESTRANSFER,ESCAMPAÑA,ESPEDIDOTELEFONICO,CODCLIENTEDESTINATARIO,CLIENTEDESTINATARIONOMBRE,CODSAPDESTINATARIO,IDCUENTADESTINATARIO,NOMBRECUENTADESTINATARIO,CODONEKEYDESTINATARIO,UBICACIONDESTINATARIO,CODPLAZAGENERAPEDIDO,RECOMMENDEDREASON,QUANTITYUSED,QUANTITYRECOMMENDED,CODONEKEY
0,12,FCH Venta Farmacias,ES-OR01359947,20220524,46226,460945,40507,NOVALAC 2 PREMIUM PROACTIVE 800G,1311186,156,NOVALAC NO FINANCIADAS,825,GR4: NOVALAC NO FIN,107.10,80.34,6.0,0.0,0.0,1,Directo,0,0,0,5757783,HEALTH AND BEAUTY ONLINE SQUARE TO,199006,1569539,PARAF. HEALTH AND BEAUTY ONLINE SQUARE TOWN HA...,WESF00005348,NaN,3363,NaN,NaN,NaN,WESF00005348
1,12,FCH Venta Farmacias,ES-OR01359947,20220524,46226,460945,27487,NOVALAC AC 800G,1307554,156,NOVALAC NO FINANCIADAS,825,GR4: NOVALAC NO FIN,134.40,107.52,6.0,0.0,0.0,1,Directo,0,0,0,5757783,HEALTH AND BEAUTY ONLINE SQUARE TO,199006,1569539,PARAF. HEALTH AND BEAUTY ONLINE SQUARE TOWN HA...,WESF00005348,NaN,3363,NaN,NaN,NaN,WESF00005348
2,12,FCH Venta Farmacias,ES-OR01359947,20220524,46226,460945,31224,UTIPRO PLUS 30 CAPS,1290954,1483,UTIPRO,826,OTROS CH,282.60,197.82,18.0,0.0,0.0,1,Directo,0,0,0,5757783,HEALTH AND BEAUTY ONLINE SQUARE TO,199006,1569539,PARAF. HEALTH AND BEAUTY ONLINE SQUARE TOWN HA...,WESF00005348,NaN,3363,NaN,NaN,NaN,WESF00005348
3,12,FCH Venta Farmacias,ES-OR01359947,20220524,46226,460945,42018,"OTC ANTIPIOJOS PERMET LOCION 1,5% 200ML",1311687,839,OTC ANTIPIOJOS,824,GR3: OTC + HIDRA,460.80,230.40,48.0,0.0,0.0,1,Directo,0,0,0,5757783,HEALTH AND BEAUTY ONLINE SQUARE TO,199006,1569539,PARAF. HEALTH AND BEAUTY ONLINE SQUARE TOWN HA...,WESF00005348,NaN,3363,NaN,NaN,NaN,WESF00005348
4,12,FCH Venta Farmacias,ES-OR01359947,20220524,46226,460945,11644,"OTC ANTIPIOJOS PERMET CHAMPU 1,5% 125ML",1303214,839,OTC ANTIPIOJOS,824,GR3: OTC + HIDRA,390.60,195.30,45.0,0.0,0.0,1,Directo,0,0,0,5757783,HEALTH AND BEAUTY ONLINE SQUARE TO,199006,1569539,PARAF. HEALTH AND BEAUTY ONLINE SQUARE TOWN HA...,WESF00005348,NaN,3363,NaN,NaN,NaN,WESF00005348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17926,12,FCH Venta Farmacias,ES-OR01350341,20220907,44681,150740,15095,GELOCATIL GRIPE PSEUDOEFEDRINA 20 COMP,1304382,172,GELOCATIL GRIPE,823,GR2: INVIERNO,179.20,107.52,28.0,0.0,0.0,3,Campaña,0,1,0,3742,ARAUJO CARBALLO EVA M. - FARMACIA,104385,232478,"FARM. ARAUJO CARBALLO, EVA MARIA",WESF02731593,Rural,56,NaN,NaN,NaN,WESF02731593
17927,12,FCH Venta Farmacias,ES-OR01372119,20220907,44681,150740,32578,GELOCATIL 1G 10 COMPRIMIDOS,1308913,136,GELOCATIL,822,GR1: GELOCATIL,37.66,37.66,14.0,0.0,0.0,3,Campaña,0,1,0,3742,ARAUJO CARBALLO EVA M. - FARMACIA,104385,232478,"FARM. ARAUJO CARBALLO, EVA MARIA",WESF02731593,Rural,56,NaN,NaN,NaN,WESF02731593
17928,12,FCH Venta Farmacias,ES-OR01372119,20220907,44681,150740,32585,GELOCATIL 1G SOLUCION ORAL 10 SOBRES,1308915,136,GELOCATIL,822,GR1: GELOCATIL,50.28,50.28,12.0,0.0,0.0,3,Campaña,0,1,0,3742,ARAUJO CARBALLO EVA M. - FARMACIA,104385,232478,"FARM. ARAUJO CARBALLO, EVA MARIA",WESF02731593,Rural,56,NaN,NaN,NaN,WESF02731593
17929,12,FCH Venta Farmacias,ES-OR01350341,20220907,44681,150740,41619,GELOCATIL 650MG 20 COMPRIMIDOS,1311580,136,GELOCATIL,822,GR1: GELOCATIL,76.80,57.60,24.0,0.0,0.0,3,Campaña,0,1,0,3742,ARAUJO CARBALLO EVA M. - FARMACIA,104385,232478,"FARM. ARAUJO CARBALLO, EVA MARIA",WESF02731593,Rural,56,NaN,NaN,NaN,WESF02731593


In [20]:
# Eliminar columnas innecesarias
df.drop(columns=['CODLINEA','NOMBRELINEA','IDBRICK','CODBRICK','CODARTICULO','NOMBREARTICULO','NOMBREPRODUCTO',
                  'NOMBREFAMILIA', 'TIPOPEDIDO','ESTRANSFER','CODCLIENTEDESTINATARIO',
                  'CLIENTEDESTINATARIONOMBRE','IDCUENTADESTINATARIO','NOMBRECUENTADESTINATARIO','CODONEKEYDESTINATARIO',
                  'CODPLAZAGENERAPEDIDO','RECOMMENDEDREASON','QUANTITYUSED','QUANTITYRECOMMENDED',
                  'UNIDADESBONIFICADAS','UNIDADESMUESTRAS','CODSAPDESTINATARIO'], inplace=True)

In [21]:
# Valores nulos
df.isna().sum()

CODPEDIDO                     0
FECHAFACTURA                  0
CODARTICULOSAP                0
CODPRODUCTO                   0
CODFAMILIA                    0
VALORBASE                     0
VALORNETO                     0
UNIDADES                      0
IDTIPOPEDIDO                  0
ESCAMPAÑA                     0
ESPEDIDOTELEFONICO            0
UBICACIONDESTINATARIO    273322
CODONEKEY                     0
dtype: int64

In [22]:
# Reemplazar valores nulos de ubicaciones
df.loc[df['UBICACIONDESTINATARIO'].isna(),'UBICACIONDESTINATARIO'] = 'Desconocido'

In [23]:
# Convert to datetime
df['FECHAFACTURA'] = pd.to_datetime(df['FECHAFACTURA'],format='%Y%m%d')
df.sort_values(by='FECHAFACTURA', ascending=True).reset_index(drop=True)

,CODPEDIDO,FECHAFACTURA,CODARTICULOSAP,CODPRODUCTO,CODFAMILIA,VALORBASE,VALORNETO,UNIDADES,IDTIPOPEDIDO,ESCAMPAÑA,ESPEDIDOTELEFONICO,UBICACIONDESTINATARIO,CODONEKEY
0,ES-OR00983309,2019-01-02,1302882,783,823,60.72,48.58,12.0,2,0,0,Residencial/Barrio Joven,WESF05920616
1,ES-OR00983410,2019-01-02,1308335,839,824,86.16,56.00,12.0,2,0,1,Residencial/Barrio Joven,WESF08031835
2,ES-OR00983210,2019-01-02,1303216,839,824,1000.98,499.82,134.0,1,0,0,Residencial/Barrio 3ª Edad,WESF09032227
3,ES-OR00983210,2019-01-02,1303218,839,824,88.60,44.30,10.0,1,0,0,Residencial/Barrio 3ª Edad,WESF09032227
4,ES-OR00983210,2019-01-02,1303208,839,824,97.50,48.75,5.0,1,0,0,Residencial/Barrio 3ª Edad,WESF09032227
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027884,ES-OR01490089,2023-06-19,1311791,1908,822,63.40,44.40,20.0,1,0,0,Residencial/Barrio Joven,WESF08030577
1027885,ES-OR01490089,2023-06-19,1310992,1908,822,22.40,15.70,5.0,1,0,0,Residencial/Barrio Joven,WESF08030577
1027886,ES-OR01490110,2023-06-19,1306641,136,822,15.20,12.15,5.0,1,0,0,Desconocido,WESF08249765
1027887,ES-OR01490089,2023-06-19,1306641,136,822,12.16,9.12,4.0,1,0,0,Residencial/Barrio Joven,WESF08030577


In [24]:
# Reemplazar el código de pedido (CODPEDIDO) por el número de pedido (ordinal)
df['NUMORDINALPEDIDO'] = df.groupby('CODONEKEY')['FECHAFACTURA'].rank(method='dense').astype(int)
df

,CODPEDIDO,FECHAFACTURA,CODARTICULOSAP,CODPRODUCTO,CODFAMILIA,VALORBASE,VALORNETO,UNIDADES,IDTIPOPEDIDO,ESCAMPAÑA,ESPEDIDOTELEFONICO,UBICACIONDESTINATARIO,CODONEKEY,NUMORDINALPEDIDO
0,ES-OR01359947,2022-05-24,1311186,156,825,107.10,80.34,6.0,1,0,0,Desconocido,WESF00005348,32
1,ES-OR01359947,2022-05-24,1307554,156,825,134.40,107.52,6.0,1,0,0,Desconocido,WESF00005348,32
2,ES-OR01359947,2022-05-24,1290954,1483,826,282.60,197.82,18.0,1,0,0,Desconocido,WESF00005348,32
3,ES-OR01359947,2022-05-24,1311687,839,824,460.80,230.40,48.0,1,0,0,Desconocido,WESF00005348,32
4,ES-OR01359947,2022-05-24,1303214,839,824,390.60,195.30,45.0,1,0,0,Desconocido,WESF00005348,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17926,ES-OR01350341,2022-09-07,1304382,172,823,179.20,107.52,28.0,3,1,0,Rural,WESF02731593,52
17927,ES-OR01372119,2022-09-07,1308913,136,822,37.66,37.66,14.0,3,1,0,Rural,WESF02731593,52
17928,ES-OR01372119,2022-09-07,1308915,136,822,50.28,50.28,12.0,3,1,0,Rural,WESF02731593,52
17929,ES-OR01350341,2022-09-07,1311580,136,822,76.80,57.60,24.0,3,1,0,Rural,WESF02731593,52


In [25]:
df.sort_values(by='FECHAFACTURA', ascending=True).reset_index(drop=True)

,CODPEDIDO,FECHAFACTURA,CODARTICULOSAP,CODPRODUCTO,CODFAMILIA,VALORBASE,VALORNETO,UNIDADES,IDTIPOPEDIDO,ESCAMPAÑA,ESPEDIDOTELEFONICO,UBICACIONDESTINATARIO,CODONEKEY,NUMORDINALPEDIDO
0,ES-OR00983309,2019-01-02,1302882,783,823,60.72,48.58,12.0,2,0,0,Residencial/Barrio Joven,WESF05920616,1
1,ES-OR00983410,2019-01-02,1308335,839,824,86.16,56.00,12.0,2,0,1,Residencial/Barrio Joven,WESF08031835,1
2,ES-OR00983210,2019-01-02,1303216,839,824,1000.98,499.82,134.0,1,0,0,Residencial/Barrio 3ª Edad,WESF09032227,1
3,ES-OR00983210,2019-01-02,1303218,839,824,88.60,44.30,10.0,1,0,0,Residencial/Barrio 3ª Edad,WESF09032227,1
4,ES-OR00983210,2019-01-02,1303208,839,824,97.50,48.75,5.0,1,0,0,Residencial/Barrio 3ª Edad,WESF09032227,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027884,ES-OR01490089,2023-06-19,1311791,1908,822,63.40,44.40,20.0,1,0,0,Residencial/Barrio Joven,WESF08030577,14
1027885,ES-OR01490089,2023-06-19,1310992,1908,822,22.40,15.70,5.0,1,0,0,Residencial/Barrio Joven,WESF08030577,14
1027886,ES-OR01490110,2023-06-19,1306641,136,822,15.20,12.15,5.0,1,0,0,Desconocido,WESF08249765,9
1027887,ES-OR01490089,2023-06-19,1306641,136,822,12.16,9.12,4.0,1,0,0,Residencial/Barrio Joven,WESF08030577,14


In [26]:
# Transformar fecha a datos en columna
df['ANOFACTURA'] = df['FECHAFACTURA'].dt.year
df['MESFACTURA'] = df['FECHAFACTURA'].dt.month
df['DIAFACTURA'] = df['FECHAFACTURA'].dt.day
df

,CODPEDIDO,FECHAFACTURA,CODARTICULOSAP,CODPRODUCTO,CODFAMILIA,VALORBASE,VALORNETO,UNIDADES,IDTIPOPEDIDO,ESCAMPAÑA,ESPEDIDOTELEFONICO,UBICACIONDESTINATARIO,CODONEKEY,NUMORDINALPEDIDO,ANOFACTURA,MESFACTURA,DIAFACTURA
0,ES-OR01359947,2022-05-24,1311186,156,825,107.10,80.34,6.0,1,0,0,Desconocido,WESF00005348,32,2022,5,24
1,ES-OR01359947,2022-05-24,1307554,156,825,134.40,107.52,6.0,1,0,0,Desconocido,WESF00005348,32,2022,5,24
2,ES-OR01359947,2022-05-24,1290954,1483,826,282.60,197.82,18.0,1,0,0,Desconocido,WESF00005348,32,2022,5,24
3,ES-OR01359947,2022-05-24,1311687,839,824,460.80,230.40,48.0,1,0,0,Desconocido,WESF00005348,32,2022,5,24
4,ES-OR01359947,2022-05-24,1303214,839,824,390.60,195.30,45.0,1,0,0,Desconocido,WESF00005348,32,2022,5,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17926,ES-OR01350341,2022-09-07,1304382,172,823,179.20,107.52,28.0,3,1,0,Rural,WESF02731593,52,2022,9,7
17927,ES-OR01372119,2022-09-07,1308913,136,822,37.66,37.66,14.0,3,1,0,Rural,WESF02731593,52,2022,9,7
17928,ES-OR01372119,2022-09-07,1308915,136,822,50.28,50.28,12.0,3,1,0,Rural,WESF02731593,52,2022,9,7
17929,ES-OR01350341,2022-09-07,1311580,136,822,76.80,57.60,24.0,3,1,0,Rural,WESF02731593,52,2022,9,7


In [27]:
# Sobreescribimos el archivo para pruebas
df.to_csv(ffile, sep=',', index=False)